---
# UGent - FEA Masters
---

### A) Fetch different masters' programme pages

In [20]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
from urllib.parse import urljoin

import re

#Base link we start scraping from
base_link = 'https://studiegids.ugent.be/2018/NL/FACULTY/E/MABA/MABA.html'

#Download html-page
html = urlopen(base_link)
soup = BeautifulSoup(html, features='lxml')

# Using CSS3 Selectors
a_tags = soup.select('div:nth-of-type(2) + ul > li.opleidingnaamlijst > a')
masters = []

#All selected a-tags contain information on a master study
for a_tag in a_tags:
    #Get relative path of master's programme page
    master_url = a_tag['href']
    #Make absolute path
    master_url = urljoin(base_link,master_url)
    #Get title of master
    master_title = a_tag.text.replace('\n','').replace('  ','').replace('-',' - ')
    #Regex for matching the master acronym
    master_acronym = re.search(r'MABA\/(.*?)\/', master_url).group(1)
    #Save for later use
    masters.append({master_acronym: {'link': master_url, 'title': master_title}})
    
    print('[%s] %s\n%s' % (master_acronym,master_title,master_url))

[EMEFUS] European Master of Science in Nuclear Fusion and Engineering Physics
https://studiegids.ugent.be/2018/NL/FACULTY/E/MABA/EMEFUS/EMEFUS.html
[EMEUPH] European Master of Science in Photonics
https://studiegids.ugent.be/2018/NL/FACULTY/E/MABA/EMEUPH/EMEUPH.html
[EMBIME] International Master of Science in Biomedical Engineering
https://studiegids.ugent.be/2018/NL/FACULTY/E/MABA/EMBIME/EMBIME.html
[EMFIRE] International Master of Science in Fire Safety Engineering
https://studiegids.ugent.be/2018/NL/FACULTY/E/MABA/EMFIRE/EMFIRE.html
[CMBIOIBE] Master of Science in Bioinformatics - afstudeerrichting Bioscience Engineering
https://studiegids.ugent.be/2018/NL/FACULTY/C/MABA/CMBIOIBE/CMBIOIBE.html
[CMBIOIEN] Master of Science in Bioinformatics - afstudeerrichting Engineering
https://studiegids.ugent.be/2018/NL/FACULTY/C/MABA/CMBIOIEN/CMBIOIEN.html
[CMBIOISB] Master of Science in Bioinformatics - afstudeerrichting Systems Biology
https://studiegids.ugent.be/2018/NL/FACULTY/C/MABA/CMBIOIS

### B) Fetch HTML partials for each master

In [21]:
soups = []
acronyms = []

#Download html-page for each master's programme page
for master in masters:
    
    acronym = list(master.keys())[0]
    base_link = master[acronym]['link']
    
    html = urlopen(base_link)
    soup = BeautifulSoup(html, features='lxml')

    #Get main body with courses
    main_iframe = soup.find_all('iframe')[0]
    #Make absolute path
    base_link= urljoin(base_link,main_iframe['src'])
    print(base_link)
    
    #Download HTML-partial with master's courses
    main_iframe_html = urlopen(base_link)
    main_iframe_soup = BeautifulSoup(main_iframe_html)
    
    #Save for later use
    soups.append(main_iframe_soup)
    acronyms.append(acronym)

    #Iterate over all nested HTML-partials
    iframes = main_iframe_soup.find_all('iframe')

    while len(iframes) != 0:
        
        #Search for all nested HTML-partials
        nested_iframes = []
        for iframe in iframes:
            src = iframe['id'] + '.html'
            
            #Make absolute path
            iframe_url = urljoin(base_link,src)
            print(iframe_url)
            
            #Download HTML-partial
            iframe_html = urlopen(iframe_url)
            iframe_soup = BeautifulSoup(iframe_html)
            
            nested_iframes.extend(iframe_soup.find_all('iframe'))
            
            #Save for later use
            soups.append(iframe_soup)
            acronyms.append(acronym)
        
        #Focus on next set of nested HTML-partials
        iframes = nested_iframes

https://studiegids.ugent.be/2018/NL/FACULTY/E/MABA/EMEFUS/EMEFUS1(0)/EMEFUS.html
https://studiegids.ugent.be/2018/NL/FACULTY/E/MABA/EMEFUS/EMEFUS1(0)/EMEFUS1.1.1.html
https://studiegids.ugent.be/2018/NL/FACULTY/E/MABA/EMEFUS/EMEFUS1(0)/EMEFUS1.1.2.html
https://studiegids.ugent.be/2018/NL/FACULTY/E/MABA/EMEFUS/EMEFUS1(0)/EMEFUS1.1.3.html
https://studiegids.ugent.be/2018/NL/FACULTY/E/MABA/EMEFUS/EMEFUS1(0)/EMEFUS1.1.4.html
https://studiegids.ugent.be/2018/NL/FACULTY/E/MABA/EMEFUS/EMEFUS1(0)/EMEFUS1.2.1.html
https://studiegids.ugent.be/2018/NL/FACULTY/E/MABA/EMEFUS/EMEFUS1(0)/EMEFUS1.2.2.html
https://studiegids.ugent.be/2018/NL/FACULTY/E/MABA/EMEFUS/EMEFUS1(0)/EMEFUS1.2.3.html
https://studiegids.ugent.be/2018/NL/FACULTY/E/MABA/EMEFUS/EMEFUS1(0)/EMEFUS1.2.4.html
https://studiegids.ugent.be/2018/NL/FACULTY/E/MABA/EMEFUS/EMEFUS1(0)/EMEFUS1.2.5.html
https://studiegids.ugent.be/2018/NL/FACULTY/E/MABA/EMEFUS/EMEFUS1(0)/EMEFUS1.1.1.1.html
https://studiegids.ugent.be/2018/NL/FACULTY/E/MABA/EMEFUS

https://studiegids.ugent.be/2018/NL/FACULTY/C/MABA/CMBIOISB/CMBIOISB4(0)/CMBIOISB4.3.1.html
https://studiegids.ugent.be/2018/NL/FACULTY/C/MABA/CMBIOISB/CMBIOISB4(0)/CMBIOISB4.3.2.html
https://studiegids.ugent.be/2018/NL/FACULTY/C/MABA/CMBIOISB/CMBIOISB4(0)/CMBIOISB4.2.1.1.html
https://studiegids.ugent.be/2018/NL/FACULTY/C/MABA/CMBIOISB/CMBIOISB4(0)/CMBIOISB4.2.1.2.html
https://studiegids.ugent.be/2018/NL/FACULTY/C/MABA/CMBIOISB/CMBIOISB4(0)/CMBIOISB4.2.1.3.html
https://studiegids.ugent.be/2018/NL/FACULTY/C/MABA/CMBIOISB/CMBIOISB4(0)/CMBIOISB4.2.1.4.html
https://studiegids.ugent.be/2018/NL/FACULTY/C/MABA/CMBIOISB/CMBIOISB4(0)/CMBIOISB4.2.1.5.html
https://studiegids.ugent.be/2018/NL/FACULTY/C/MABA/CMBIOISB/CMBIOISB4(0)/CMBIOISB4.2.1.6.html
https://studiegids.ugent.be/2018/NL/FACULTY/C/MABA/CMBIOISB/CMBIOISB4(0)/CMBIOISB4.2.1.7.html
https://studiegids.ugent.be/2018/NL/FACULTY/E/MABA/EMBIEN/EMBIEN8(0)/EMBIEN.html
https://studiegids.ugent.be/2018/NL/FACULTY/E/MABA/EMBIEN/EMBIEN8(0)/EMBIEN8.

https://studiegids.ugent.be/2018/NL/FACULTY/E/MABA/EMELMEEE/EMELMEEE6(0)/EMELMEEE6.3.1.2.2.html
https://studiegids.ugent.be/2018/NL/FACULTY/E/MABA/EMELMEEE/EMELMEEE6(0)/EMELMEEE6.3.3.1.1.html
https://studiegids.ugent.be/2018/NL/FACULTY/E/MABA/EMELMEEE/EMELMEEE6(0)/EMELMEEE6.3.3.1.2.html
https://studiegids.ugent.be/2018/NL/FACULTY/E/MABA/EMELMEEE/EMELMEEE6(0)/EMELMEEE6.3.3.1.3.html
https://studiegids.ugent.be/2018/NL/FACULTY/E/MABA/EMELMEEE/EMELMEEE6(0)/EMELMEEE6.3.3.1.4.html
https://studiegids.ugent.be/2018/NL/FACULTY/E/MABA/EMELMEEE/EMELMEEE6(0)/EMELMEEE6.3.3.1.5.html
https://studiegids.ugent.be/2018/NL/FACULTY/E/MABA/EMELMEEE/EMELMEEE6(0)/EMELMEEE6.3.3.1.6.html
https://studiegids.ugent.be/2018/NL/FACULTY/E/MABA/EMELMEEE/EMELMEEE6(0)/EMELMEEE6.3.3.1.7.html
https://studiegids.ugent.be/2018/NL/FACULTY/E/MABA/EMELMEEE/EMELMEEE6(0)/EMELMEEE6.3.3.1.8.html
https://studiegids.ugent.be/2018/NL/FACULTY/E/MABA/EMELMEEE/EMELMEEE6(0)/EMELMEEE6.3.3.1.9.html
https://studiegids.ugent.be/2018/NL/FACU

https://studiegids.ugent.be/2018/NL/FACULTY/E/MABA/EMELMERA/EMELMERA6(0)/EMELMERA.html
https://studiegids.ugent.be/2018/NL/FACULTY/E/MABA/EMELMERA/EMELMERA6(0)/EMELMERA6.3.1.html
https://studiegids.ugent.be/2018/NL/FACULTY/E/MABA/EMELMERA/EMELMERA6(0)/EMELMERA6.3.2.html
https://studiegids.ugent.be/2018/NL/FACULTY/E/MABA/EMELMERA/EMELMERA6(0)/EMELMERA6.3.3.html
https://studiegids.ugent.be/2018/NL/FACULTY/E/MABA/EMELMERA/EMELMERA6(0)/EMELMERA6.3.1.1.html
https://studiegids.ugent.be/2018/NL/FACULTY/E/MABA/EMELMERA/EMELMERA6(0)/EMELMERA6.3.1.2.html
https://studiegids.ugent.be/2018/NL/FACULTY/E/MABA/EMELMERA/EMELMERA6(0)/EMELMERA6.3.1.3.html
https://studiegids.ugent.be/2018/NL/FACULTY/E/MABA/EMELMERA/EMELMERA6(0)/EMELMERA6.3.2.1.html
https://studiegids.ugent.be/2018/NL/FACULTY/E/MABA/EMELMERA/EMELMERA6(0)/EMELMERA6.3.2.2.html
https://studiegids.ugent.be/2018/NL/FACULTY/E/MABA/EMELMERA/EMELMERA6(0)/EMELMERA6.3.2.3.html
https://studiegids.ugent.be/2018/NL/FACULTY/E/MABA/EMELMERA/EMELMERA6(0)/

### C) Fetch PDFs (studiefiches) for each _unique_ course

In [22]:
import os
from google.cloud import translate

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "./GoogleTranslateCred.json"
translate_client = translate.Client(target_language='en')

courses_information = {}
pdf_urls = []

#Scrape course links/pdfs of each master's programme HTML-partial
for ids,soup in enumerate(soups):
    
    #Get a-tags representing PDF course descriptions
    course_pdfs = soup.select('a[href$=".pdf"]')
    
    #Get other course information in bulk (for all)
    semester_values = soup.select('tr.rowclass > td.semester')
    lecturers = soup.select('tr.rowclass > td.lesgever')
    contact_hours = soup.select('tr.rowclass > td.contacturen')
    study_time = soup.select('tr.rowclass > td.studietijd')
    credits = soup.select('tr.rowclass > td.studiepunten')

    #Iterate over all course description links
    for idx, course_pdf in enumerate(course_pdfs):
        
        #We assume that thesis isn't done on Erasmus
        if 'Masterproef' not in course_pdf.text:
            
            #Make absolute path
            course_pdf_url = urljoin(base_link,course_pdf['href'])
            #Regex for extracting the course's tag/acronym
            tag = re.search('studiefiches\/(.*?)\.pdf', course_pdf_url).group(1)
            
            #If tag isn't known, create new entry
            if not tag in list(courses_information.keys()):
                
                #We want a translated course name
                course_name = translate_client.translate(course_pdf.text)['translatedText']
                
                #Save PDF link for later use
                pdf_urls.append(course_pdf_url)
                
                #Create new entry for tag
                courses_information[tag]={'name': course_name, 
                                         'semester': semester_values[idx].text, 
                                         'lecturer': lecturers[idx].text, 
                                         'contact_hours': contact_hours[idx].text,
                                         'study_time': study_time[idx].text,
                                         'credits': credits[idx].text,
                                         'link': course_pdf_url,
                                         'masters': [acronyms[ids]]}
                
                print("%s (%s): \n%s" % (course_name, tag, course_pdf_url))
            
            #Tag already appeared in other master -> just update 'masters' field
            else:
                courses_information[tag]['masters'].append(acronyms[ids])

Joint Experimentation and Analysis Session (E901190): 
https://studiegids.ugent.be/2018/NL/studiefiches/E901190.pdf
Joint Practicum (CEA, Cadarache) (E901194): 
https://studiegids.ugent.be/2018/NL/studiefiches/E901194.pdf
Statistical Physics (E900016): 
https://studiegids.ugent.be/2018/NL/studiefiches/E900016.pdf
Materials Physics (E900196): 
https://studiegids.ugent.be/2018/NL/studiefiches/E900196.pdf
Advanced Atomic and Molecular Physics (E900028): 
https://studiegids.ugent.be/2018/NL/studiefiches/E900028.pdf
Physics of Plasma Propulsion  (E901191): 
https://studiegids.ugent.be/2018/NL/studiefiches/E901191.pdf
Applied electromagnetism (E900039): 
https://studiegids.ugent.be/2018/NL/studiefiches/E900039.pdf
Computational solutions of golf problems (E022700): 
https://studiegids.ugent.be/2018/NL/studiefiches/E022700.pdf
Atomic and molecular physics (E025010): 
https://studiegids.ugent.be/2018/NL/studiefiches/E025010.pdf
Plasmafysica (E026220): 
https://studiegids.ugent.be/2018/NL/studi

Research practice in photonics (E030721): 
https://studiegids.ugent.be/2018/NL/studiefiches/E030721.pdf
Optical communication systems (E012420): 
https://studiegids.ugent.be/2018/NL/studiefiches/E012420.pdf
Sensors and microsystem electronics (E030940): 
https://studiegids.ugent.be/2018/NL/studiefiches/E030940.pdf
Physics of semiconductor technologies and components (E031521): 
https://studiegids.ugent.be/2018/NL/studiefiches/E031521.pdf
Innovation in photonics (E030770): 
https://studiegids.ugent.be/2018/NL/studiefiches/E030770.pdf
Current developments in photonics (E030740): 
https://studiegids.ugent.be/2018/NL/studiefiches/E030740.pdf
Photonics (E030620): 
https://studiegids.ugent.be/2018/NL/studiefiches/E030620.pdf
Design of refractive and diffractive optical imaging systems (E030961): 
https://studiegids.ugent.be/2018/NL/studiefiches/E030961.pdf
Optical spectroscopy of materials (E027300): 
https://studiegids.ugent.be/2018/NL/studiefiches/E027300.pdf
Display technology (E032411): 

Fundamentals of Pathophysiology & Diagnostic Methods (E900345): 
https://studiegids.ugent.be/2018/NL/studiefiches/E900345.pdf
Practical Training Placement (E900346): 
https://studiegids.ugent.be/2018/NL/studiefiches/E900346.pdf
Biomechanics (E900314): 
https://studiegids.ugent.be/2018/NL/studiefiches/E900314.pdf
Bioinstrumentation (E900315): 
https://studiegids.ugent.be/2018/NL/studiefiches/E900315.pdf
Basic Medical Sciences (E900316): 
https://studiegids.ugent.be/2018/NL/studiefiches/E900316.pdf
Biomaterials (E900317): 
https://studiegids.ugent.be/2018/NL/studiefiches/E900317.pdf
Project/Clinical Placement (E900323): 
https://studiegids.ugent.be/2018/NL/studiefiches/E900323.pdf
Bioelectromagnetism (E022250): 
https://studiegids.ugent.be/2018/NL/studiefiches/E022250.pdf
Biomedical acustics (E022030): 
https://studiegids.ugent.be/2018/NL/studiefiches/E022030.pdf
Artificial intelligence (E016330): 
https://studiegids.ugent.be/2018/NL/studiefiches/E016330.pdf
Stage 1 [en, nl] (E099960): 


Clinical Neural Engineering (E900325): 
https://studiegids.ugent.be/2018/NL/studiefiches/E900325.pdf
Implantable Neural Systems (E900326): 
https://studiegids.ugent.be/2018/NL/studiefiches/E900326.pdf
Modeling in medicine and biomedical engineering: cases (E092913): 
https://studiegids.ugent.be/2018/NL/studiefiches/E092913.pdf
Numerical flow mechanics (E040520): 
https://studiegids.ugent.be/2018/NL/studiefiches/E040520.pdf
Computational biomechanics [en, nl] (E092891): 
https://studiegids.ugent.be/2018/NL/studiefiches/E092891.pdf
From medical image to computer model [en, nl] (E092922): 
https://studiegids.ugent.be/2018/NL/studiefiches/E092922.pdf
Biomedical Robotics (E900430): 
https://studiegids.ugent.be/2018/NL/studiefiches/E900430.pdf
Virtual Reality (E900431): 
https://studiegids.ugent.be/2018/NL/studiefiches/E900431.pdf
Design of microsystems [nl] (E030900): 
https://studiegids.ugent.be/2018/NL/studiefiches/E030900.pdf
Sensors and actuators (E008445): 
https://studiegids.ugent.be/

Parallel and distributed software systems (E017930): 
https://studiegids.ugent.be/2018/NL/studiefiches/E017930.pdf
Computational Challenges in Bioinformatics (C003711): 
https://studiegids.ugent.be/2018/NL/studiefiches/C003711.pdf
Optimization techniques (E004120): 
https://studiegids.ugent.be/2018/NL/studiefiches/E004120.pdf
Cellular and Molecular Biology (C003712): 
https://studiegids.ugent.be/2018/NL/studiefiches/C003712.pdf
Introduction to Bioinformatics (C003713): 
https://studiegids.ugent.be/2018/NL/studiefiches/C003713.pdf
Parallel computer systems (E034140): 
https://studiegids.ugent.be/2018/NL/studiefiches/E034140.pdf
Information security (E019400): 
https://studiegids.ugent.be/2018/NL/studiefiches/E019400.pdf
Softwarearchitectuur (E017822): 
https://studiegids.ugent.be/2018/NL/studiefiches/E017822.pdf
Softwarebeheer [nl]  (E017840): 
https://studiegids.ugent.be/2018/NL/studiefiches/E017840.pdf
Programming languages (E017421): 
https://studiegids.ugent.be/2018/NL/studiefiches/

Psychology of adolescence [nl] (H001784): 
https://studiegids.ugent.be/2018/NL/studiefiches/H001784.pdf
Introduction to human resource management (E076130): 
https://studiegids.ugent.be/2018/NL/studiefiches/E076130.pdf
Sustainable energy and rational energy use (E039060): 
https://studiegids.ugent.be/2018/NL/studiefiches/E039060.pdf
Coaching and diversity [nl] (H001977): 
https://studiegids.ugent.be/2018/NL/studiefiches/H001977.pdf
Models and methods in operational research (E004255): 
https://studiegids.ugent.be/2018/NL/studiefiches/E004255.pdf
Project management (E076820): 
https://studiegids.ugent.be/2018/NL/studiefiches/E076820.pdf
Business skills [en, nl] (F000551): 
https://studiegids.ugent.be/2018/NL/studiefiches/F000551.pdf
Chemistry of nanostructures (C002963): 
https://studiegids.ugent.be/2018/NL/studiefiches/C002963.pdf
Chemical modification of renewable raw materials (I000530): 
https://studiegids.ugent.be/2018/NL/studiefiches/I000530.pdf
Electrochemistry [en] (E070900): 
h

Ship behavior in shallow and limited water (E055320): 
https://studiegids.ugent.be/2018/NL/studiefiches/E055320.pdf
Design of multimedia applications (E017920): 
https://studiegids.ugent.be/2018/NL/studiefiches/E017920.pdf
Research project (E031710): 
https://studiegids.ugent.be/2018/NL/studiefiches/E031710.pdf
Design project (E033710): 
https://studiegids.ugent.be/2018/NL/studiefiches/E033710.pdf
Mobile and broadband access networks (E012320): 
https://studiegids.ugent.be/2018/NL/studiefiches/E012320.pdf
Queance analysis and simulation (E011322): 
https://studiegids.ugent.be/2018/NL/studiefiches/E011322.pdf
Advanced modulation and coding (E012210): 
https://studiegids.ugent.be/2018/NL/studiefiches/E012210.pdf
Advanced multimedia applications (E019161): 
https://studiegids.ugent.be/2018/NL/studiefiches/E019161.pdf
Analog electronics [nl] (E030210): 
https://studiegids.ugent.be/2018/NL/studiefiches/E030210.pdf
Compilers [nl]  (E018520): 
https://studiegids.ugent.be/2018/NL/studiefiches/

Construction techniques (E056600): 
https://studiegids.ugent.be/2018/NL/studiefiches/E056600.pdf
Gas turbines (E037621): 
https://studiegids.ugent.be/2018/NL/studiefiches/E037621.pdf
Electrical drive technology [nl] (E036110): 
https://studiegids.ugent.be/2018/NL/studiefiches/E036110.pdf
Climatisation and cooling technology (E039211): 
https://studiegids.ugent.be/2018/NL/studiefiches/E039211.pdf
Hydrostatics and propulsion of maritime constructions (E055270): 
https://studiegids.ugent.be/2018/NL/studiefiches/E055270.pdf
General view, structure and construction of maritime constructions (E055030): 
https://studiegids.ugent.be/2018/NL/studiefiches/E055030.pdf
Introduction to maritime technology (E055045): 
https://studiegids.ugent.be/2018/NL/studiefiches/E055045.pdf
Calculation of structural constructions I (E044120): 
https://studiegids.ugent.be/2018/NL/studiefiches/E044120.pdf
Maneuvering and seagoing behavior of maritime constructions (E055290): 
https://studiegids.ugent.be/2018/NL/st

### D) Parse PDFs + extract content for indexing

In [94]:
import os
from google.cloud import translate

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "./GoogleTranslateCred.json"
translate_client = translate.Client(target_language='en')

def dutch_check(content):
    if ' de ' in content or ' het ' in content or ' voor ' in content or ' een ' in content:
        return translate_client.translate(content)['translatedText']
    return ''

In [95]:
## REMARK ##
# Tika server depends on Java JAXB API, which is removed from Java JDK 9+ (no longer default class path).
# Make sure openjdk in conda env. is set to version 8 (tested with 8/0.152)
from tika import parser

from bs4 import BeautifulSoup, NavigableString

for idp, pdf_url in enumerate(pdf_urls):
    # Parse data from PDF
    pdf = parser.from_file(pdf_url, xmlContent=True)

    content = []

    # Format as XML, so we can use tree structure
    soup = BeautifulSoup(pdf['content'], features='lxml')
    
    for child in soup.body.descendants:
        if isinstance(child, NavigableString):
            #remove unwanted chars and redundant whitespaces
            child = child.replace(u'\xa0', u' ').replace('\n',' ').replace('•',' ')
            child = re.sub(' +',' ',child).strip()
            if child:
                content.append(child)

    #Fetch course's tag/acronym
    tag = re.search('studiefiches\/(.*?)\.pdf', pdf_url).group(1)

    #There is very little structure in the PDF/XML tree
    #We base on position of definition of course's language
    keywords_index = -1
    if "English" in content:
        keywords_index = content.index("English") + 1
        courses_information[tag]['language'] = 'EN'
    elif "Dutch" in content:
        keywords_index = content.index("Dutch") + 1
        courses_information[tag]['language'] = 'NL'
    elif "Engels" in content:
        keywords_index = content.index("Engels") + 1
        courses_information[tag]['language'] = 'EN'
    elif "Nederlands" in content:
        keywords_index = content.index("Nederlands") + 1
        courses_information[tag]['language'] = 'NL'
    elif "Dutch, English" in content:
        keywords_index = content.index("Dutch, English") + 1
        courses_information[tag]['language'] = 'EN'
    elif "Nederlands, Engels" in content:
        keywords_index = content.index("Nederlands, Engels") + 1
        courses_information[tag]['language'] = 'EN'
        
    #Most of the times, the PDF's aren't filled in :-(
    #We can detect unfilled info by dirtily checking on occurence of 'ContacturenStudietijd' or '2(Goedgekeurd)'
    if not ('ContacturenStudietijd' in content[keywords_index] or '2(Goedgekeurd)' in  content[keywords_index]):
        #Check if translation needed, returns translation if so
        translated = dutch_check(content[keywords_index])
        keywords = translated if translated else content[keywords_index]
        
        #Update dictionary fields
        courses_information[tag]['keywords'] = keywords.replace(', ',',')
        
        if not 'ContacturenStudietijd' in content[keywords_index+1]:
            #Check if translation needed, returns translation if so
            translated = dutch_check(content[keywords_index+1])
            situating = translated if translated else content[keywords_index+1]
            
            #Update dictionary fields
            courses_information[tag]['situating'] = situating
            
            if not 'ContacturenStudietijd' in content[keywords_index+2]:
                #Check if translation needed, returns translation if so
                translated = dutch_check(content[keywords_index+2])
                cont = translated if translated else content[keywords_index+2]
                
                #Update dictionary fields
                courses_information[tag]['content'] = cont
     
    #PDF isn't filled in
    else:
        courses_information[tag]['keywords'] = ''
        courses_information[tag]['situating'] = ''
        courses_information[tag]['content'] = ''   

    #Print output
    print('•••PDF••• #%d' % (idp))
    print('[%s] %s %s\n%s\n\n%s\n\n%s\n' % (tag, courses_information[tag]['name'], courses_information[tag]['masters'],
                                         courses_information[tag]['keywords'], courses_information[tag]['situating'],
                                         courses_information[tag]['content']))

2019-01-11 08:58:42,057 [MainThread  ] [INFO ]  Retrieving https://studiegids.ugent.be/2018/NL/studiefiches/E901190.pdf to C:\Users\Thibo\AppData\Local\Temp/https-studiegids-ugent-be-2018-nl-studiefiches-e901190-pdf.
2019-01-11 08:58:42,236 [MainThread  ] [INFO ]  Retrieving https://studiegids.ugent.be/2018/NL/studiefiches/E901194.pdf to C:\Users\Thibo\AppData\Local\Temp/https-studiegids-ugent-be-2018-nl-studiefiches-e901194-pdf.
2019-01-11 08:58:42,395 [MainThread  ] [INFO ]  Retrieving https://studiegids.ugent.be/2018/NL/studiefiches/E900016.pdf to C:\Users\Thibo\AppData\Local\Temp/https-studiegids-ugent-be-2018-nl-studiefiches-e900016-pdf.


•••PDF••• #0
[E901190] Joint Experimentation and Analysis Session ['EMEFUS']
Plasma physic,tokamak experiment,plasma diagnostics

The EMTRAIC is a training school to get practical experience with tokamak operation. The school is held on the COMPASS tokamak at the Institute of Plasma Physics, Academy of Sciences of the Czech Republic (IPP Prague). The main goal is to acquaint participants with many aspects of experimental plasma physics on a tokamak. planning of experiment participation in measurements data processing discussion of results within the experimental group presentation of achieved results at the closing workshop preparation a report Just one day is devoted to introductory lectures.

Students are distributed in several groups, which are supervised by experts from the Tokamak Department of the Institute of Plasma Physics. Each group is composed of 1- 2 students. Tasks to be executed are formulated according current experimental program on the COMPASS tokamak. For example (in 

2019-01-11 08:58:42,571 [MainThread  ] [INFO ]  Retrieving https://studiegids.ugent.be/2018/NL/studiefiches/E900196.pdf to C:\Users\Thibo\AppData\Local\Temp/https-studiegids-ugent-be-2018-nl-studiefiches-e900196-pdf.


•••PDF••• #2
[E900016] Statistical Physics ['EMEFUS']
Statistical mechanics,canonical ensemble,macrocanonical ensemble,phase transitions,critical phenomena,Langevin equation,Boltzmann equation,transport.

Develop intuitive pictures of the micro- and the macroscopic world. Distinguish between equilibrium and non-equilibrium states. Understand the statistical origin of thermodynamic potentials. Calculate the partition function of simple systems. Apply mean-field theories to a variety of systems. Understand criticality and universality. Use transport equations.

Foundations: the microscopic and macroscopic world, ergodic hypothesis, the micro- canonical ensemble. Canonical ensemble: derivation, thermodynamic potentials, fluctuations, applications. Bose-Einstein gas: Bose-Einstein condensation, examples. Fermi gas: Fermi distribution and Fermi energy, examples. Phase transitions and critical phenomena: the Ising model, Van der Waals theory of liquids, critical phenomena, universality. Non-

2019-01-11 08:58:42,759 [MainThread  ] [INFO ]  Retrieving https://studiegids.ugent.be/2018/NL/studiefiches/E900028.pdf to C:\Users\Thibo\AppData\Local\Temp/https-studiegids-ugent-be-2018-nl-studiefiches-e900028-pdf.
2019-01-11 08:58:42,937 [MainThread  ] [INFO ]  Retrieving https://studiegids.ugent.be/2018/NL/studiefiches/E901191.pdf to C:\Users\Thibo\AppData\Local\Temp/https-studiegids-ugent-be-2018-nl-studiefiches-e901191-pdf.
2019-01-11 08:58:43,086 [MainThread  ] [INFO ]  Retrieving https://studiegids.ugent.be/2018/NL/studiefiches/E900039.pdf to C:\Users\Thibo\AppData\Local\Temp/https-studiegids-ugent-be-2018-nl-studiefiches-e900039-pdf.


...

•••PDF••• #620
[E064881] Application of Technical Textiles ['EMTEXT']
Technical textiles,filtration,medical,automotive,protection,industrial,geotechnical.

Technical applications of textile materials.

Filtration textiles Protective Textiles Medical textiles Civil Engineering Applications Industrial Garment Applications Transportation Applications (automotive, aviation)

•••PDF••• #621
[E064891] Technical Textile Manufacturing Technology ['EMTEXT']
Manufacturing,weaving,knitting,braiding processes,testing,properties,structures,constructions,applications

Concentrating on manufacturing technical textiles is a key issue.

PART A All textiles are made of fibres or yarns. For load-bearing technical textiles, special products with special mechanical properties are used. Typical examples are viscose, polyester and polyamide, aramide, carbon, glass and metal fibres and yarns. 1 Introduction 1 Man-made fibre spinning processes 1 Types of fibres and yarns 1 Market overview 2 Man-made fibres fro

2019-01-11 09:09:22,044 [MainThread  ] [INFO ]  Retrieving https://studiegids.ugent.be/2018/NL/studiefiches/E064910.pdf to C:\Users\Thibo\AppData\Local\Temp/https-studiegids-ugent-be-2018-nl-studiefiches-e064910-pdf.
2019-01-11 09:09:22,205 [MainThread  ] [INFO ]  Retrieving https://studiegids.ugent.be/2018/NL/studiefiches/E064701.pdf to C:\Users\Thibo\AppData\Local\Temp/https-studiegids-ugent-be-2018-nl-studiefiches-e064701-pdf.


•••PDF••• #622
[E064691] Innovative Methods for the Product Development Process for Garments and Technical Applications in the Ready-Made Industry ['EMTEXT']
CAD/CAM systems in the apparel industry,computer aided clothing design and manufacturing,integrating CAD/CAM with other IT-systems,3 dimensional design,material behaviour

To illustrate the importance of CAD/CAM systems.

CAD/CAM Systems in the Apparel Industry Computer Aided Clothing Design and Manufacturing Integrating CAD/CAM with other IT-systems 3 Dimensional Design for Clothes and Technical Textiles Material Behaviour

•••PDF••• #623
[E064910] Comfort of Textiles ['EMTEXT']
Comfort in textiles,physiological comfort,thermal comfort,sensorial comfort,ergonomic comfort (what makes fabrics comfortable?) different methods of measuring comfort in humans,laboratory methods of fabrics and clothing assessment.

To extend the knowledge in the field of human comfort wearing different kinds of clothing.

Definition of comfort, types of 

2019-01-11 09:09:22,363 [MainThread  ] [INFO ]  Retrieving https://studiegids.ugent.be/2018/NL/studiefiches/E064930.pdf to C:\Users\Thibo\AppData\Local\Temp/https-studiegids-ugent-be-2018-nl-studiefiches-e064930-pdf.
2019-01-11 09:09:22,526 [MainThread  ] [INFO ]  Retrieving https://studiegids.ugent.be/2018/NL/studiefiches/E064871.pdf to C:\Users\Thibo\AppData\Local\Temp/https-studiegids-ugent-be-2018-nl-studiefiches-e064871-pdf.


•••PDF••• #624
[E064701] Intelligent Textiles ['EMTEXT']
smart textiles,interactive textiles,sensoring,data processing,actuation,storage,communication,concepts,release mechanisms,micro-encapsulation,electrotextiles,optical fibres

To obtain an understanding of functional textile materials.

Definition, evolution and functions of Smart Textiles Electroconductive textiles Sensors: electrical, optical Actuators: mechanical, chemical, thermal, optical, electrical Energy: batteries, energy scavenging (from heat, light, deformation) Communication: textile antennae Components of smart textiles, textile integration

•••PDF••• #625
[E064930] Sustainability in Textiles ['EMTEXT']
Environmental impact,ecological / environmental terms and definitions,environmental tools,critical processes and substances,optimisation

Sustainable design and manufacturing in a broad context

Textiles in an Environmental Perspective Environmental Tools REACH Analysis of Typical Textile Processes and Downstream Treatm

In [96]:
#Some postprocessing for altering semester values and translation
for tag in courses_information.keys():
    
    sem = courses_information[tag]['semester']
    
    if 'A:1\nB:1' in sem:
        courses_information[tag]['semester'] = '1'
    elif 'A:1\nB:2' in sem:
        courses_information[tag]['semester'] = 'J'
    elif 'A:2\nB:2' in sem:
        courses_information[tag]['semester'] = '2'
    elif 'A:J\nB:1' in sem:
        courses_information[tag]['semester'] = 'J'
    elif 'A:J\nB:J' in sem:
        courses_information[tag]['semester'] = 'J'
    elif '1a' in sem:
        courses_information[tag]['semester'] = '1'
    elif '2a' in sem:
        courses_information[tag]['semester'] = '2'
        
    #Since our dutch_check always failed (ofcourse, keywords don't have common dutch words in them)
    #Always translate keywords (easier here than rerunning previous PDF content extraction)
    courses_information[tag]['keywords'] = translate_client.translate(courses_information[tag]['keywords'])['translatedText']
    
print('Done!')

Done!


### E) Pickle for later use

In [97]:
import os, sys, pickle

if not os.path.exists("coursesUG"):
    os.mkdir("coursesUG")

for tag in list(courses_information.keys()):
    with open('./coursesUG/%s.pkl' % tag, 'wb') as f:
        pickle.dump(courses_information[tag], f, pickle.HIGHEST_PROTOCOL)
        
print('Pickled! :-)')

Pickled! :-)


---
# KULeuven
---

### A) Fetch different masters' information pages

In [1]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
from urllib.parse import urljoin

import re

#Base link we start scraping from
base_link = 'https://onderwijsaanbod.kuleuven.be/opleidingen/e/'

#Download html-page
html = urlopen(base_link)
soup = BeautifulSoup(html, features='lxml')

# Using CSS3 Selectors
# 0031 = Master's
a_tags = soup.select('div#content div.columnright > div:nth-of-type(2) div.0031 a')
masters = []

#All selected a-tags contain information on a master study
for a_tag in a_tags:
    #Get relative path of master's information page
    master_url = a_tag['href']
    #Make absolute path
    master_url = urljoin(base_link,master_url)
    #Get title of master
    master_title = a_tag.text.replace('\n','')
    
    # The joint master in Energy does not have information on its courses
    if "EIT-KIC" not in master_title:
        masters.append(master_url)
        print('%s\n%s' % (master_title,master_url))

Master of Biomedical Engineering (Leuven)
https://onderwijsaanbod.kuleuven.be/opleidingen/e/CQ_51360389.htm
Master of Chemical Engineering (Leuven)
https://onderwijsaanbod.kuleuven.be/opleidingen/e/CQ_51370025.htm
Master of Engineering: Computer Science (Leuven)
https://onderwijsaanbod.kuleuven.be/opleidingen/e/CQ_52364384.htm
Master of Electrical Engineering (Leuven)
https://onderwijsaanbod.kuleuven.be/opleidingen/e/CQ_50657365.htm
Master of Engineering: Energy (Leuven)
https://onderwijsaanbod.kuleuven.be/opleidingen/e/CQ_51384404.htm
Master of Materials Engineering (Leuven)
https://onderwijsaanbod.kuleuven.be/opleidingen/e/CQ_50545818.htm
Master of Engineering: Logistics and Traffic (Leuven)
https://onderwijsaanbod.kuleuven.be/opleidingen/e/CQ_53845475.htm
Master of Mechanical Engineering (Leuven)
https://onderwijsaanbod.kuleuven.be/opleidingen/e/CQ_52354207.htm
Master of Mathematical Engineering (Leuven)
https://onderwijsaanbod.kuleuven.be/opleidingen/e/CQ_52357094.htm
Master of Nan

### B) Fetch masters' programme pages

In [2]:
soups = []
acronyms = []

#Download html-page for each master's information page
for master_link in masters:  
    html = urlopen(master_link)
    soup = BeautifulSoup(html, features='lxml')
    
    #Get relative path of master's programme
    a_tag = soup.select('div#content div#oase_heading_programmas a')
    programme_link = a_tag[0]['href']
    
    #Make absolute path
    programme_link= urljoin(master_link,programme_link)
    
    #Download html-page for master's programme page
    html = urlopen(programme_link)
    soup = BeautifulSoup(html, features='lxml')

    #Save for later use
    soups.append(soup)
    
    #Acronym of master is in master's information page link
    master_acronym = re.search(r'\/e\/(.*?)\.htm', master_link).group(1)
    
    #Save for later use
    acronyms.append(master_acronym)
    
    print("%s: %s" % (master_acronym,programme_link))

CQ_51360389: https://onderwijsaanbod.kuleuven.be/opleidingen/e/SC_51360430.htm
CQ_51370025: https://onderwijsaanbod.kuleuven.be/opleidingen/e/SC_51370066.htm
CQ_52364384: https://onderwijsaanbod.kuleuven.be/opleidingen/e/SC_52364422.htm
CQ_50657365: https://onderwijsaanbod.kuleuven.be/opleidingen/e/SC_51520356.htm
CQ_51384404: https://onderwijsaanbod.kuleuven.be/opleidingen/e/SC_51384442.htm
CQ_50545818: https://onderwijsaanbod.kuleuven.be/opleidingen/e/SC_52927450.htm
CQ_53845475: https://onderwijsaanbod.kuleuven.be/opleidingen/e/SC_53840435.htm
CQ_52354207: https://onderwijsaanbod.kuleuven.be/opleidingen/e/SC_52354240.htm
CQ_52357094: https://onderwijsaanbod.kuleuven.be/opleidingen/e/SC_52357132.htm
CQ_50269006: https://onderwijsaanbod.kuleuven.be/opleidingen/e/SC_51926826.htm
CQ_51228258: https://onderwijsaanbod.kuleuven.be/opleidingen/e/SC_54299343.htm
CQ_50269018: https://onderwijsaanbod.kuleuven.be/opleidingen/e/SC_51016883.htm
CQ_50550147: https://onderwijsaanbod.kuleuven.be/opl

### C) Fetch _unique_ course pages

In [6]:
import os
from google.cloud import translate

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "./GoogleTranslateCred.json"
translate_client = translate.Client(target_language='en')

courses_information = {}
course_links = []

#Scrape course links of each master's programme page
for ids,soup in enumerate(soups):
    #Get a-tags representing specific course
    course_refs = soup.select('a[href^="/syllabi/e/"]')
    
    #Iterate over different courses contained in master's programme
    for idx, course_ref in enumerate(course_refs):
        
        #We assume that thesis isn't done on Erasmus
        if "Master's Thesis" not in course_ref.text:
            
            #Make absolute path
            course_link = urljoin(masters[ids],course_ref['href'])
            #Regex for extracting the course's tag/acronym
            tag = re.search('syllabi\/e\/(.*?)\.htm', course_link).group(1)
            
            #If tag isn't known, create new entry
            if not tag in list(courses_information.keys()):
                
                #We want a translated course name
                course_name = translate_client.translate(course_ref.text)['translatedText']
                
                #Create new entry for tag
                courses_information[tag]={'name': course_name,
                                          'link': course_link,
                                         'masters': [acronyms[ids]]}
                #Save course link for later use
                course_links.append(course_link)
                
                print("%s (%s): \n%s" % (course_name, tag, course_link))
            
            #Tag already appeared in other master -> just update 'masters' field
            else:
                courses_information[tag]['masters'].append(acronyms[ids])

Functional Anatomy of the Human Locomotor System (H9X22AE): 
https://onderwijsaanbod.kuleuven.be/syllabi/e/H9X22AE.htm
Human Cell Biology (H03H0BE): 
https://onderwijsaanbod.kuleuven.be/syllabi/e/H03H0BE.htm
Human System Physiology (H03I4BE): 
https://onderwijsaanbod.kuleuven.be/syllabi/e/H03I4BE.htm
Biomedical Measurements and Stimulation (H03H7AE): 
https://onderwijsaanbod.kuleuven.be/syllabi/e/H03H7AE.htm
Biomedical Data Processing (H03I2AE): 
https://onderwijsaanbod.kuleuven.be/syllabi/e/H03I2AE.htm
Biomaterials I (H03U4AE): 
https://onderwijsaanbod.kuleuven.be/syllabi/e/H03U4AE.htm
Musculoskeletal Biomechanics (H03H1AE): 
https://onderwijsaanbod.kuleuven.be/syllabi/e/H03H1AE.htm
Medical Imaging and Analysis (H03H5AE): 
https://onderwijsaanbod.kuleuven.be/syllabi/e/H03H5AE.htm
Biofluid Mechanics (H03J2AE): 
https://onderwijsaanbod.kuleuven.be/syllabi/e/H03J2AE.htm
Medical Equipment and Regulatory Affairs (H03I5AE): 
https://onderwijsaanbod.kuleuven.be/syllabi/e/H03I5AE.htm
Design i

Modelling of Complex Systems (H0N05AE): 
https://onderwijsaanbod.kuleuven.be/syllabi/e/H0N05AE.htm
Distributed Systems (H0N08AE): 
https://onderwijsaanbod.kuleuven.be/syllabi/e/H0N08AE.htm
Software Architecture (H07Z9BE): 
https://onderwijsaanbod.kuleuven.be/syllabi/e/H07Z9BE.htm
Fundamentals for Computer Science (H07Z7AE): 
https://onderwijsaanbod.kuleuven.be/syllabi/e/H07Z7AE.htm
Comparative Programming Languages (H0S01AE): 
https://onderwijsaanbod.kuleuven.be/syllabi/e/H0S01AE.htm
Fundamentals of Artificial Intelligence (H02A0AE): 
https://onderwijsaanbod.kuleuven.be/syllabi/e/H02A0AE.htm
Introduction to ICT Security (H0N21AE): 
https://onderwijsaanbod.kuleuven.be/syllabi/e/H0N21AE.htm
Development of Secure Software (H04K5AE): 
https://onderwijsaanbod.kuleuven.be/syllabi/e/H04K5AE.htm
Security of Network and Computer Infrastructure (H04G4AE): 
https://onderwijsaanbod.kuleuven.be/syllabi/e/H04G4AE.htm
Capita Selecta Computer Science: Secure Software (H05M8BE): 
https://onderwijsaanbo

Electromobility (H04C8AE): 
https://onderwijsaanbod.kuleuven.be/syllabi/e/H04C8AE.htm
Industrial Automation and Control (H04D0AE): 
https://onderwijsaanbod.kuleuven.be/syllabi/e/H04D0AE.htm
Measurement and Testing, including High Voltages (H04D2AE): 
https://onderwijsaanbod.kuleuven.be/syllabi/e/H04D2AE.htm
Light and Lighting Systems (H04D4AE): 
https://onderwijsaanbod.kuleuven.be/syllabi/e/H04D4AE.htm
Electromagnetic Processing of Materials (H04B1AE): 
https://onderwijsaanbod.kuleuven.be/syllabi/e/H04B1AE.htm
Flexible Electrical Power Systems (H0T47AE): 
https://onderwijsaanbod.kuleuven.be/syllabi/e/H0T47AE.htm
Aerodynamics (H04O1AE): 
https://onderwijsaanbod.kuleuven.be/syllabi/e/H04O1AE.htm
Numerical Techniques in Fluid Dynamics (H04U8AE): 
https://onderwijsaanbod.kuleuven.be/syllabi/e/H04U8AE.htm
Turbulence (H04Y7AE): 
https://onderwijsaanbod.kuleuven.be/syllabi/e/H04Y7AE.htm
Aircraft Engines (H04Z8AE): 
https://onderwijsaanbod.kuleuven.be/syllabi/e/H04Z8AE.htm
Physics of Nuclear R

Finite Elements, Part 1: Frame Structures (H9X20AE): 
https://onderwijsaanbod.kuleuven.be/syllabi/e/H9X20AE.htm
Simulation Theory and Applications (D0R19AE): 
https://onderwijsaanbod.kuleuven.be/syllabi/e/D0R19AE.htm
Data Mining and Neural Networks (H03V7BE): 
https://onderwijsaanbod.kuleuven.be/syllabi/e/H03V7BE.htm
Fundamentals of Artificial Intelligence (H02A0DE): 
https://onderwijsaanbod.kuleuven.be/syllabi/e/H02A0DE.htm
Maintenance Management (H00M4AE): 
https://onderwijsaanbod.kuleuven.be/syllabi/e/H00M4AE.htm
Modelling Land Use Changes (G0I83AE): 
https://onderwijsaanbod.kuleuven.be/syllabi/e/G0I83AE.htm
Optimization: Special Topics (D0M90BE): 
https://onderwijsaanbod.kuleuven.be/syllabi/e/D0M90BE.htm
Numerical Modelling in Mechanical Engineering (H04U3AE): 
https://onderwijsaanbod.kuleuven.be/syllabi/e/H04U3AE.htm
Theory of Elasticity and Plasticity (H03Y1AE): 
https://onderwijsaanbod.kuleuven.be/syllabi/e/H03Y1AE.htm
Integrated Project (H04P9AE): 
https://onderwijsaanbod.kuleu

Nanobiology (I0P50CE): 
https://onderwijsaanbod.kuleuven.be/syllabi/e/I0P50CE.htm
Biomachines and Biomimetics (I0O79BE): 
https://onderwijsaanbod.kuleuven.be/syllabi/e/I0O79BE.htm
Physical Chemistry of Biological Systems (G0G71AE): 
https://onderwijsaanbod.kuleuven.be/syllabi/e/G0G71AE.htm
Biofluidics (I0P05CE): 
https://onderwijsaanbod.kuleuven.be/syllabi/e/I0P05CE.htm
Micro- and Nanofluidics (H06X4BE): 
https://onderwijsaanbod.kuleuven.be/syllabi/e/H06X4BE.htm
Photophysics and Photochemistry of Molecular Materials (H06D5AE): 
https://onderwijsaanbod.kuleuven.be/syllabi/e/H06D5AE.htm
Electrochemical Methods in Inorganic Chemistry (G0J65BE): 
https://onderwijsaanbod.kuleuven.be/syllabi/e/G0J65BE.htm
Structures and Microstructures of Materials (H08Z2AE): 
https://onderwijsaanbod.kuleuven.be/syllabi/e/H08Z2AE.htm
Electronic Structure of Molecular Materials (H06B8AE): 
https://onderwijsaanbod.kuleuven.be/syllabi/e/H06B8AE.htm
Materials Characterisation Techniques I: Chemical Analyses and 

Nanooptics (H09O8AE): 
https://onderwijsaanbod.kuleuven.be/syllabi/e/H09O8AE.htm
Magnetism on the Nanoscale (H09O9AE): 
https://onderwijsaanbod.kuleuven.be/syllabi/e/H09O9AE.htm
Biofunctionalised Surfaces (H00P1AE): 
https://onderwijsaanbod.kuleuven.be/syllabi/e/H00P1AE.htm
Diffraction Methods (H00P2AE): 
https://onderwijsaanbod.kuleuven.be/syllabi/e/H00P2AE.htm
Materials for Nanoelectronics and Vacuum Technology (H0S18AE): 
https://onderwijsaanbod.kuleuven.be/syllabi/e/H0S18AE.htm
Physical Characterization of Organic and Inorganic Thin Films (H0S19AE): 
https://onderwijsaanbod.kuleuven.be/syllabi/e/H0S19AE.htm
Electromechanical Networks (H0S20AE): 
https://onderwijsaanbod.kuleuven.be/syllabi/e/H0S20AE.htm
Concepts of Molecular Modelling (H0S22AE): 
https://onderwijsaanbod.kuleuven.be/syllabi/e/H0S22AE.htm
Biological Hydrodynamics (H0S23AE): 
https://onderwijsaanbod.kuleuven.be/syllabi/e/H0S23AE.htm
Computer Arithmetic (H0T16AE): 
https://onderwijsaanbod.kuleuven.be/syllabi/e/H0T16AE.h

Regression Analysis (G0S75AE): 
https://onderwijsaanbod.kuleuven.be/syllabi/e/G0S75AE.htm
Analysis of Variance (G0S76AE): 
https://onderwijsaanbod.kuleuven.be/syllabi/e/G0S76AE.htm
Generalized Linear Models (G0A18AE): 
https://onderwijsaanbod.kuleuven.be/syllabi/e/G0A18AE.htm
Epidemiology (G0B73AE): 
https://onderwijsaanbod.kuleuven.be/syllabi/e/G0B73AE.htm
Statistical Consulting (G0B66AE): 
https://onderwijsaanbod.kuleuven.be/syllabi/e/G0B66AE.htm
Concepts of Clinical Trials (G0B69AE): 
https://onderwijsaanbod.kuleuven.be/syllabi/e/G0B69AE.htm
Concepts of Bayesian Data Analysis (G0B74AE): 
https://onderwijsaanbod.kuleuven.be/syllabi/e/G0B74AE.htm
Concepts of Multilevel, Longitudinal and Mixed Models (G0B76AE): 
https://onderwijsaanbod.kuleuven.be/syllabi/e/G0B76AE.htm
Advanced Non-Parametric Statistics (G0A23AE): 
https://onderwijsaanbod.kuleuven.be/syllabi/e/G0A23AE.htm
Advanced Analytics in a Big Data World (D0S06AE): 
https://onderwijsaanbod.kuleuven.be/syllabi/e/D0S06AE.htm
Optimi

### D) Fetch content from course page and put in dictionary

In [12]:
import os
from bs4 import NavigableString
from google.cloud import translate

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "./GoogleTranslateCred.json"
translate_client = translate.Client(target_language='en')

for course_link in course_links:
    
    #Download HTML of course page
    html = urlopen(course_link)
    soup = BeautifulSoup(html, features='lxml')
    
    #Regex for extracting course's tag/acronym
    tag = re.search('syllabi\/e\/(.*?)\.htm', course_link).group(1)
    
    #Scrape core description of course
    content_div = soup.select_one('div[id^="tab_inhoud"]')
    aims_div = soup.select_one('div[id^="tab_doelstellingen"]')
    
    #Do some preprocessing steps on the descriptions
    content = ''
    aims = ''
    
    #Some courses might not have a proper content description
    if content_div:
        for child in content_div.descendants:
            if isinstance(child, NavigableString) and not child == NavigableString("Content"):
                content = content + child + ' '
                
    #Some courses might not have proper aims          
    if aims_div:            
        for child in aims_div.descendants:
            if isinstance(child, NavigableString) and not child == NavigableString("Aims"):
                aims = aims + child + ' '
            
    #Remove unwanted chars -> u'\xa0' is &nbsp;
    content = content.replace('•',' ').replace('–',' ').replace('*',' ').replace('- ',' ').replace(u'\xa0',' ')
    aims = aims.replace('•',' ').replace('–',' ').replace('*',' ').replace('- ',' ').replace(u'\xa0',' ')
    
    #Replace Escape Sequences ...        
    content = content.replace('\n',' ').replace('\t',' ').strip()
    aims = aims.replace('\n',' ').replace('\t',' ').strip()
    # ... and redundant whitespace
    content = re.sub(' +',' ',content)
    aims = re.sub(' +',' ',aims)

    #Remove rare occurence of <!-- --> comment in text
    try:
        html_comment = re.search('(<!--.*?-->)',content).group(1)
        if html_comment:
            content = content.replace(html_comment,'').strip()
    except:
        pass
    
    #Lastly we want to translate sporadic dutch descriptions
    if ' de ' in content or ' het ' in content or ' voor ' in content or ' een ' in content:
        content = translate_client.translate(content)['translatedText']
    if ' de ' in aims or ' het ' in aims or ' voor ' in aims or ' een ' in aims:
        aims = translate_client.translate(aims)['translatedText']

    #Print output
    print('[%s]\n%s\n\n%s\n' % (tag,content,aims))
         
    #Save information in dictionary
    courses_information[tag]['content'] = content
    courses_information[tag]['situating'] = aims
    
    #Extract semester
    semester = soup.select_one('span.semester')
    if semester:
        courses_information[tag]['semester'] = \
        ('1' if 'First' in semester.text else ('J' if 'Both' in semester.text else '2'))
    
    #Extract lecturer
    lecturer = soup.select_one('span.docent > a')
    if lecturer:
        courses_information[tag]['lecturer'] = lecturer.text
    
    #Extract ECTS credits and study time
    credits = soup.select_one('span.studiepunten')
    if credits:
        courses_information[tag]['credits'] = credits.text.split(' ')[0]
        courses_information[tag]['study_time'] = str(float(courses_information[tag]['credits'])*30)
    
    #Extract contant hours
    contact_hours = soup.select('span.duur > a')
    if contact_hours:
        courses_information[tag]['contact_hours'] = contact_hours[1].text
    
    #Extract language
    language = soup.select_one('span.taal')
    if language:
        courses_information[tag]['language'] = ('EN' if 'English' in language.text else 'NL')


[H9X22AE]
In this course the following items will be treated: An introduction in which the student is made familiar with the anatomical terminology. A general anatomical part describing terminology on the structure of bones, joints and muscles. Description of the trunk, specifically describing the spine, chest, abdomen and pelvis. Description of the upper extremity, specifically describing the bones, joints, muscles, the vasculature and the nerves. Description of the lower extremity, specifically describing the bones, joints, muscles, the vasculature and the nerves.

The intention is to systematically describe the motion system of the human anatomy, to describe the bones, joints and muscles per segment of the human body. To analyze each joint, its movements and actions, as well as an introductory look at how functional joints are used and how injuries can occur, mentioning, in nutshell or briefly, what can go wrong within the human locomotoric system or in other words some general path

[H03U4AE]
The course introduces first the conditions that non-viable biomaterials have to fulfill to be used as biomaterial. In the next part the specific material families such as metals, ceramics and polymers are discussed in more details with special attention to specific materials in every family. In a third part lessons are replaced by seminars given by invited specialists applying those materials daily: orthopedics, hart valves, traumatology, tissue engineering, artificial skin, dental applications.

To know about the properties (potentials and limits) of non-viable biomaterials such as polymers, ceramics and metallic alloys, which are used as implant materials or used in contact with living tissue. 1. Every material has physical, chemical and mechanical properties. The student is able to evaluate which properties deserve special attention at particular applications . 2. The students can evaluate the host environment and can select the material according to the environment. 3. Th

[H02D1AE]
Introduction and situation of the course The basic genetic algorithm Related methodologies, e.g. genetic programming, evolution strategies, ... More details on genetic operators (selection, cross-over, mutation) and on the representation of the population for some model problems Theoretical foundation: schemata theorem; strategies to avoid premature convergence and to improve convergence or execution time: sampling mechanisms, adaptation of the fitness function, varying population size, ... Handling constraints Comparison with other methods for search and optimisation: gradient-based methods (hill climbing), simulated annealing, tabu search Model problems: travelling salesman problem, transportation problem Case studies: e.g. timetabling, concept learning, classification, evolving 3D morphology and behaviour by competition Software packages

The aim of the course is to describe and to analyse genetic algorithms and (other) evolution strategies in sufficient detail, such that 

[H06W1AE]
Information on the Lectures. In biomedical data processing, the aim is to extract clinically, biochemically or pharmaceutically relevant information (e.g rhythms or spikes) in terms of parameters out of low-quality measurements in order to enable an improved medical diagnosis. Typically, biomedical data are affected by large measurement errors, largely due to the noninvasive nature of the measurement process or the severe constraints to keep the input signal as low as possible for safety and bio-ethical reasons. Accurate and automated quantification of this information requires an ingenious combination of the following 4 issues: an adequate pretreatment of the data, the design of an appropriate model and model validation, a fast and numerically robust model parameter quantification method, and an extensive evaluation study, using in-vivo and patient data, up to the embedding of the advanced tools into userfriendly user interfaces to be used by clinicians. To solve each of the

[H0T59AE]
Lecture topics: Cells and proteins Protein/material interactions Tissues and extracellular matrix Inflammation Immunity and the complement system The foreign body response Blood/materials interactions Biofilm formation Biocompatibility testing The use of standards in biomaterials research In vitro testing In vivo testing

The student is able to explain basic mechanisms of the host response (wound healing, tissue regeneration, immune response, genetic influence, etc.) in response to biomaterials. The student can propose appropriate in vitro and in vivo tests to evaluate the biocompatibility and functionality of a material intended for medical use.

[H02C6AE]
Topics covered include (not necessarily in this order): 1) Data mining overview 2) The data mining process 3) Recommender systems 4) Association rule mining 5) Sequential pattern mining 6) Clustering 7) Large scale decision tree learning 8) Advanced topics on ensemble methods 9) Using unlabeled data 10) Data streams 11) Ad

...

[P0Q45AE]
The course is divided into two parts: 1. Bayesian parameter estimation and prediction 2. Case studies focusing on cognitive models

After completing this OPO the student: is acquainted with the Bayesian approach in quantitative psychology. is able to build their own Bayesian models. is able to apply these to their own data. is able to draw their own conclusions.

[P0Q48BE]
The content of the seminar changes every year.

Thirteen universities (the K.U.Leuven, together with the Universities of Amsterdam, Padua, Oldenburg, Debrecen, Graz, Complutense of Madrid, Lisbon, Tartu, Tubingen, Oulu, and the Internation/Jacobs University of Bremen) from eleven European countries have joined forces to create an EU-funded Intensive Programme on Quantitative Psychology. If you enroll in the course “European seminar on Quantitative psychology”, you may participate in the Intensive Programme. The rationale and objectives of the European seminar are three-fold. (1) In the psychological science

[G9X29AE]
Preparatory reading to the exercise sessions

The student must understand basic and more advanced techniques of neural networks for datamining, as well as related methods of nonlinear modeling. The student must be able to apply the methods to real data sets and constructively work towards good solutions.

[D0S91AE]
Content of the course: Introduction to Monte Carlo Simulatntion and to Bootstrapping using OLS IV estimation Binary response models Order response models Censored regression and selection models Count data models Introduction to panel data estimation Advanced panel data estimation In order to highlight important assumptions of the econometric models and possible pitfalls when working with real data, we will either work with simulated data or real data. Simulated data have the advantage that one can specify a "true" data generating process and verify if the econometric methods reover the true model parameters. Possibly more interesting is the fact that we can violat

[D0N56AE]
Part 0 : refreshing basic concepts of life insurance mathematics; the random future lifetime; valuation of life insurance benefits (basics); valuation of life annuity benefits (basics); demographic markers (life expectancy etc.); premium calculation; policy values. Part I : stochastic mortality models and projected life tables; parametric mortality models; smoothing of mortality data; forecasting techniques; Lee-Carter and LifeMetrics models: specication, calibration, simulation; calculation of demographic markers and insurance products using projected mortality rates; discussion of model selection and model risk; discussion of basis risk and models for portfolio specific mortality. Part II : multistate models for life and health insurance; examples of multistate models; assumptions and notations; probabilities: formulae + numerical evaluation; premiums; policy values; multiple decrement models; joint life and last survivor models; transitions at specified ages. Part III: sel

[N0A05AE]
The aim of this course is to achieve an understanding of why and how the EU acts in the context of global governance. Introducing international relations literature on globalization and global governance, we will try to understand which strategies, norms and interests drive the EU's external action in various global governance fora and policy areas, e.g. the United Nations, the WTO, the G7/G8/G20, global security governance, global economic governance, global aid, etc. During the course, we will look at the EU’s external action and representation in the context of formal and informal global governance arenas, such as the United Nations, International Monetary Fund, and the G7, G8 and G20. Furthermore, the EU’s relations with other regions and transnational actors in global governance will be addressed and critically evaluated. Finally, the course focuses on how global external perceptions of the EU either confirm or contradict the Union’s objectives and self-perception in the

[G0M92AE]
Principles of geospatial data modelling: Entity-Relationship modelling and Visual Modelling by means of the Unified Modelling Language Relational database design and relational database management systems Object-relational database management systems Introduction to the Structured Query Language

Become knowledgeable regarding concepts of geographic information science and geospatial database technology for modelling geographic reality in the context of geographic information systems Acquire a capability to apply the related techniques independently Become knowledgeable about spatial database technology and its relationships with Geographic Information Systems for modelling of Geographic Reality Acquiring skills related to working with and managing (object-)relational databases using PostgreSQL/PostGIS-software by means of an elaborated tutorial Applying skills related to working with and managing (object-)relational databases using PostgreSQL/PostGIS-software through a set o

### E) Pickle for later use

In [13]:
import os, sys, pickle

if not os.path.exists("courses"):
    os.mkdir("courses")

for tag in list(courses_information.keys()):
    with open('./courses/%s.pkl' % tag, 'wb') as f:
        pickle.dump(courses_information[tag], f, pickle.HIGHEST_PROTOCOL)
        
print('Pickled! :-)')

Pickled! :-)


---
# VUB
---

In [14]:
#REMARK
#Since the titles of the masters don't really match with VUB's study guide,
#we're going to pragmatically predefine the masters according to the study guide.
#But they can be scraped from: http://www.vub.ac.be/en/study/masters#faculty- under Faculty of Engineering
master_titles = ['European Master of Photonics', 'Master of Biomedical Engineering', 
                'Master of Applied Sciences and Engineering: Applied Computer Science', 
                'Master in Applied Sciences and Engineering: Computer Science', 'Master of Architectural Engineering', 
                'Master of Chemical and Materials Engineering', 'Master of Civil Engineering', 
                'Master of Electrical Engineering', 'Master of Electromechanical Engineering', 
                'Master of Physical Land Resources', 'Master of Nuclear Engineering']

#Let us immediately define our own acronyms for the masters, since VUB doesn't provide any.
#Also assume Applied Computer Science = Computer Science (probably Informatics at other universities)
acronyms = ['VUBPho', 'VUBBioMed', 'VUBCs', 'VUBArch', 'VUBChem', 'VUBCivil', 'VUBElec', 'VUBElecMech', 'VUBLand', 'VUBNucl']

### A) Fetch masters' progamme pages

In [15]:
from bs4 import BeautifulSoup
from urllib.request import urlopen, Request
from urllib.parse import urljoin

import re

#Continue scraping at study guide
base_link = 'https://caliweb.cumulus.vub.ac.be/caliweb/'

#Spoofing the header
hdr = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11'}

#Download HTML-page
req = Request(base_link, headers=hdr)
html = urlopen(req)
soup = BeautifulSoup(html, features='lxml')

# Using CSS3 Selectors
a_tags = soup.select('table')[1].select('a')

masters = []
soups = []
acronyms_map = []

#Loop over a-tags, which contains links to masters' information pages
for a_tag in a_tags:
    
    #Only if one of the predefined masters
    if a_tag.text in master_titles:
        
        #Get link to master information page
        master_url = a_tag['href']
        #Make absolute path
        master_url = urljoin(base_link,master_url)
        
        #Download HTML-page of master information
        req = Request(master_url, headers=hdr)
        html = urlopen(req)
        soup = BeautifulSoup(html, features='lxml')
        
        a_partims = soup.select('a')
        
        #Since some masters offer (multiple or none) partims/tracks and any lack of structure in HTML -> dirty scraping
        #Select all unclassed a_tags between most bottom backtotop-class a_tags
        for a_partim in a_partims[::-1][1:]:
            try: 
                if 'backtotop' in a_partim['class']:
                    break
            except:
                #Link to traject programme of master
                master_url_partim = a_partim['href']
                #Make absolute path
                master_url_partim = urljoin(master_url,master_url_partim)
            
            #Open master traject's programme page
            req = Request(master_url_partim, headers=hdr)
            html = urlopen(req)
            master_soup = BeautifulSoup(html, features='lxml')
            
            #Save for later use
            masters.append(master_url_partim)
            soups.append(master_soup)
            
            #Get index of correct acronym
            acr_ind = master_titles.index(a_tag.text)
            acr_ind = acr_ind if acr_ind<3 else acr_ind-1
            
            #Save for acronym for later use
            acronyms_map.append(acronyms[acr_ind])
            
            print('[%s] - %s - %s:\n%s' % (acronyms[acr_ind],a_tag.text,a_partim.text,master_url_partim))


[VUBPho] - European Master of Photonics - Standaard traject:
https://caliweb.cumulus.vub.ac.be/caliweb/?page=plan&id=00232&anchor=0000000558&target=pr&year=1819&language=en&output=html
[VUBCs] - Master in Applied Sciences and Engineering: Computer Science - Profile Web & Information Systems:
https://caliweb.cumulus.vub.ac.be/caliweb/?page=plan&id=00262&anchor=0000000616&target=pr&year=1819&language=en&output=html
[VUBCs] - Master in Applied Sciences and Engineering: Computer Science - Profile Software Languages and Software Engineering:
https://caliweb.cumulus.vub.ac.be/caliweb/?page=plan&id=00262&anchor=0000000615&target=pr&year=1819&language=en&output=html
[VUBCs] - Master in Applied Sciences and Engineering: Computer Science - Profile Profile Multimedia:
https://caliweb.cumulus.vub.ac.be/caliweb/?page=plan&id=00262&anchor=0000001713&target=pr&year=1819&language=en&output=html
[VUBCs] - Master in Applied Sciences and Engineering: Computer Science - Profile Multimedia:
https://caliweb

### B) Fetch _unique_ course pages

In [16]:
courses_information = {}
course_links = []

#Scrape course links of each master's programme page
for ids,soup in enumerate(soups):
    #Get a-tags representing specific course
    course_refs = soup.select('table a')
    
    #Iterate over different courses contained in master's programme
    for idx, course_ref in enumerate(course_refs):
        
        #We assume that thesis/external course isn't done on Erasmus
        if "Thesis" not in course_ref.text and "External" not in course_ref.text:
            
            #Make absolute path
            course_link = urljoin(masters[ids],course_ref['href'])
            #Regex for extracting the course's name
            course_name = re.search('(?<=credits - )(.*?)$', course_ref.text).group(1)
            
            #Intermediate page to access course page
            req = Request(course_link, headers=hdr)
            html = urlopen(req)
            course_soup = BeautifulSoup(html, features='lxml')
            
            #Get a-tag linking to course page
            course = course_soup.select_one('table a')
            #Make absolute path
            course_link = urljoin(course_link,course['href'])
            #Regex for extracting the course's tag/acronym
            tag = re.search('(?<=catalog number )(.*?)(?= for)', course.text).group(1)
            
            #If tag isn't known, create new entry
            if not tag in list(courses_information.keys()):
                
                #Create new entry for tag
                courses_information[tag]={'name': course_name,
                                          'link': course_link,
                                         'masters': [acronyms_map[ids]]}
                
                #Save course link for later use
                course_links.append(course_link)
                
                print("%s (%s): \n%s" % (course_name, tag, course_link))
                
            #Tag already appeared in other master -> just update 'masters' field
            else:
                courses_information[tag]['masters'].append(acronyms_map[ids])
                

Introduction to Business (4019187DEW): 
https://caliweb.cumulus.vub.ac.be/caliweb/?page=course-offer&id=006817&anchor=1&target=pr&year=1819&language=en&output=html
Lasers (4020324ENR): 
https://caliweb.cumulus.vub.ac.be/caliweb/?page=course-offer&id=009365&anchor=1&target=pr&year=1819&language=en&output=html
Optical Materials (4020325ENR): 
https://caliweb.cumulus.vub.ac.be/caliweb/?page=course-offer&id=009366&anchor=1&target=pr&year=1819&language=en&output=html
Photonics (4017310DNR): 
https://caliweb.cumulus.vub.ac.be/caliweb/?page=course-offer&id=006779&anchor=1&target=pr&year=1819&language=en&output=html
Mathematics in Photonics (4012212ENR): 
https://caliweb.cumulus.vub.ac.be/caliweb/?page=course-offer&id=004655&anchor=1&target=pr&year=1819&language=en&output=html
Laboratories in Photonics Research (4016690ENR): 
https://caliweb.cumulus.vub.ac.be/caliweb/?page=course-offer&id=007213&anchor=1&target=pr&year=1819&language=en&output=html
Business Aspects of Micro-electronics and Phot

Biomechanics (4017088DNR): 
https://caliweb.cumulus.vub.ac.be/caliweb/?page=course-offer&id=006758&anchor=1&target=pr&year=1819&language=en&output=html
Biomedical Imaging (4017091DNR): 
https://caliweb.cumulus.vub.ac.be/caliweb/?page=course-offer&id=006759&anchor=1&target=pr&year=1819&language=en&output=html
Innovation Management (4017219EER): 
https://caliweb.cumulus.vub.ac.be/caliweb/?page=course-offer&id=007466&anchor=1&target=pr&year=1819&language=en&output=html
Marketing (1021457ANR): 
https://caliweb.cumulus.vub.ac.be/caliweb/?page=course-offer&id=010256&anchor=1&target=pr&year=1819&language=en&output=html
Technological Business Development Project (4015734ENR): 
https://caliweb.cumulus.vub.ac.be/caliweb/?page=course-offer&id=006533&anchor=1&target=pr&year=1819&language=en&output=html
Technological Business Development Project (4015735ENR): 
https://caliweb.cumulus.vub.ac.be/caliweb/?page=course-offer&id=006534&anchor=1&target=pr&year=1819&language=en&output=html
IP Management an

Processus dynamiques (4014872FNR): 
https://caliweb.cumulus.vub.ac.be/caliweb/?page=course-offer&id=006143&anchor=1&target=pr&year=1819&language=en&output=html
Decision Engineering (4014870FNR): 
https://caliweb.cumulus.vub.ac.be/caliweb/?page=course-offer&id=006141&anchor=1&target=pr&year=1819&language=en&output=html
Artificial Intelligence Programming Paradigms (4013069FNR): 
https://caliweb.cumulus.vub.ac.be/caliweb/?page=course-offer&id=005055&anchor=1&target=pr&year=1819&language=en&output=html
Advanced Methods in Bioinformatics (4018221FNR): 
https://caliweb.cumulus.vub.ac.be/caliweb/?page=course-offer&id=005472&anchor=1&target=pr&year=1819&language=en&output=html
Swarm Intelligence (4014868FNR): 
https://caliweb.cumulus.vub.ac.be/caliweb/?page=course-offer&id=006139&anchor=1&target=pr&year=1819&language=en&output=html
Adaptive Systems Seminar (4018115ENR): 
https://caliweb.cumulus.vub.ac.be/caliweb/?page=course-offer&id=005057&anchor=1&target=pr&year=1819&language=en&output=html

Communication Networks: Protocols and Architectures (4016975ENR): 
https://caliweb.cumulus.vub.ac.be/caliweb/?page=course-offer&id=007386&anchor=1&target=pr&year=1819&language=en&output=html
Telecommunication Networks (4016446ENR): 
https://caliweb.cumulus.vub.ac.be/caliweb/?page=course-offer&id=007046&anchor=1&target=pr&year=1819&language=en&output=html
Project Computer Engineering (4016450EER): 
https://caliweb.cumulus.vub.ac.be/caliweb/?page=course-offer&id=007050&anchor=1&target=pr&year=1819&language=en&output=html
Image Processing (4010976ENR): 
https://caliweb.cumulus.vub.ac.be/caliweb/?page=course-offer&id=004210&anchor=1&target=pr&year=1819&language=en&output=html
Business Aspects of Software Industry (4018228ENR): 
https://caliweb.cumulus.vub.ac.be/caliweb/?page=course-offer&id=006818&anchor=1&target=pr&year=1819&language=en&output=html
Entrepreneurship (4016998EER): 
https://caliweb.cumulus.vub.ac.be/caliweb/?page=course-offer&id=007409&anchor=1&target=pr&year=1819&language=e

Construction (4017454DER): 
https://caliweb.cumulus.vub.ac.be/caliweb/?page=course-offer&id=007589&anchor=1&target=pr&year=1819&language=en&output=html
Bio-electronics (4017596DNR): 
https://caliweb.cumulus.vub.ac.be/caliweb/?page=course-offer&id=006768&anchor=1&target=pr&year=1819&language=en&output=html
Medical Physics (4017254EER): 
https://caliweb.cumulus.vub.ac.be/caliweb/?page=course-offer&id=006761&anchor=1&target=pr&year=1819&language=en&output=html
Biomedical Product Development (4016105FER): 
https://caliweb.cumulus.vub.ac.be/caliweb/?page=course-offer&id=006769&anchor=1&target=pr&year=1819&language=en&output=html
Medical Equipment (4016095FNR): 
https://caliweb.cumulus.vub.ac.be/caliweb/?page=course-offer&id=006760&anchor=1&target=pr&year=1819&language=en&output=html
Technology in Clinical Neuroscience (4018366FNR): 
https://caliweb.cumulus.vub.ac.be/caliweb/?page=course-offer&id=007788&anchor=1&target=pr&year=1819&language=en&output=html
Human and Environment,  Safety and R

Molecular structural characterization and analysis (4016478ENR): 
https://caliweb.cumulus.vub.ac.be/caliweb/?page=course-offer&id=007078&anchor=1&target=pr&year=1819&language=en&output=html
Parameter Estimation and Modelling (4016476ENR): 
https://caliweb.cumulus.vub.ac.be/caliweb/?page=course-offer&id=007076&anchor=1&target=pr&year=1819&language=en&output=html
Microstructural design and characterization of inorganic materials (4016481ENR): 
https://caliweb.cumulus.vub.ac.be/caliweb/?page=course-offer&id=007081&anchor=1&target=pr&year=1819&language=en&output=html
Organic Chemistry: Reactions and Mechanisms (4016479ENR): 
https://caliweb.cumulus.vub.ac.be/caliweb/?page=course-offer&id=007079&anchor=1&target=pr&year=1819&language=en&output=html
Polymer Materials (4015821ENR): 
https://caliweb.cumulus.vub.ac.be/caliweb/?page=course-offer&id=006578&anchor=1&target=pr&year=1819&language=en&output=html
Unit operations (4016475ENR): 
https://caliweb.cumulus.vub.ac.be/caliweb/?page=course-offe

Research Methods in Civil Engineering (4020675ENR): 
https://caliweb.cumulus.vub.ac.be/caliweb/?page=course-offer&id=009644&anchor=1&target=pr&year=1819&language=en&output=html
Form-finding and Structural Optimisation (4017557FNR): 
https://caliweb.cumulus.vub.ac.be/caliweb/?page=course-offer&id=007025&anchor=1&target=pr&year=1819&language=en&output=html
Advanced Computational Structural Mechanics (4016429FNR): 
https://caliweb.cumulus.vub.ac.be/caliweb/?page=course-offer&id=007029&anchor=1&target=pr&year=1819&language=en&output=html
Geology and engineering geology (4017285FNR): 
https://caliweb.cumulus.vub.ac.be/caliweb/?page=course-offer&id=007507&anchor=1&target=pr&year=1819&language=en&output=html
Mechanics of Geomaterials (4017284FNR): 
https://caliweb.cumulus.vub.ac.be/caliweb/?page=course-offer&id=007506&anchor=1&target=pr&year=1819&language=en&output=html
Rock mechanics and underground constructions (4018726FNR): 
https://caliweb.cumulus.vub.ac.be/caliweb/?page=course-offer&id=

Railway Technology (4020271FNR): 
https://caliweb.cumulus.vub.ac.be/caliweb/?page=course-offer&id=009342&anchor=1&target=pr&year=1819&language=en&output=html
Vehicle Electronics (4016376ENR): 
https://caliweb.cumulus.vub.ac.be/caliweb/?page=course-offer&id=006990&anchor=1&target=pr&year=1819&language=en&output=html
Vehicle Dynamics and Kinematics (4016313FNR): 
https://caliweb.cumulus.vub.ac.be/caliweb/?page=course-offer&id=006933&anchor=1&target=pr&year=1819&language=en&output=html
Programmable Logic Controllers (4016350FNR): 
https://caliweb.cumulus.vub.ac.be/caliweb/?page=course-offer&id=006970&anchor=1&target=pr&year=1819&language=en&output=html
Experimental Fluid Mechanics (4011237FNR): 
https://caliweb.cumulus.vub.ac.be/caliweb/?page=course-offer&id=004333&anchor=1&target=pr&year=1819&language=en&output=html
Global navigation satellite systems (4012983FER): 
https://caliweb.cumulus.vub.ac.be/caliweb/?page=course-offer&id=005016&anchor=1&target=pr&year=1819&language=en&output=html

Aircraft Systems (4016337FER): 
https://caliweb.cumulus.vub.ac.be/caliweb/?page=course-offer&id=006957&anchor=1&target=pr&year=1819&language=en&output=html
Aircraft Specification and Certification (4016339FNR): 
https://caliweb.cumulus.vub.ac.be/caliweb/?page=course-offer&id=006959&anchor=1&target=pr&year=1819&language=en&output=html
Computational Fluid Dynamics (4016336FNR): 
https://caliweb.cumulus.vub.ac.be/caliweb/?page=course-offer&id=006956&anchor=1&target=pr&year=1819&language=en&output=html
Introduction to nuclear physics and measurements (6009268FNR): 
https://caliweb.cumulus.vub.ac.be/caliweb/?page=course-offer&id=003594&anchor=1&target=pr&year=1819&language=en&output=html
Nuclear energy: introduction (6009267FNR): 
https://caliweb.cumulus.vub.ac.be/caliweb/?page=course-offer&id=003593&anchor=1&target=pr&year=1819&language=en&output=html
Nuclear fuel cycle (6009278FER): 
https://caliweb.cumulus.vub.ac.be/caliweb/?page=course-offer&id=003599&anchor=1&target=pr&year=1819&langua

### C) Fetch content from course page and put in dictionary

In [17]:
for course_link in course_links:
    
    #Download HTML of course page
    req = Request(course_link, headers=hdr)
    html = urlopen(req)
    soup = BeautifulSoup(html, features='lxml')
    
    #first 2 p-tags contain course info on tag, credits and study
    p_tags = soup.select('div.content p')[:2]
    
    #Regexs for extracting course information
    tag = re.search('(?<=catalog number )(.*?)(?= for)', p_tags[1].text).group(1)
    credits = re.search('^(.*?)(?= ECTS)',p_tags[0].text).group(1)
    study = re.search('(?<=credits)(.*?)(?= h study time)',p_tags[0].text).group(1)
    
    ### Instead of using .text -> better with loop over decendants (no explicit delete <br/> req.), but this also works :-)
    
    #Instead of searching for tags for remainder of info -> infeasible because lack of structure in HTML docs
    #Take all TEXT from content-tag and preprocess it to make searching with regex possible.
    body = soup.select_one('dl.dl-horizontal')
    #Replace <br/> with single whitespace
    body = str(body).replace('<br/>',' ')
    body = BeautifulSoup(body, features='lxml')
    #Replace Escape Sequences and redundant whitespace
    body_text = body.text.replace('\n',' ').replace('\t',' ').replace(u'\xa0',' ').strip()
    body_text = re.sub(' +', ' ',body_text)
    
    
    #Get rest of info by regex
    sem_value = re.search('(?<=Semester )(.*?)(?= semester)',body_text).group(1)
    language = re.search('(?<=Taught in )(.*?)(?= Faculty)',body_text).group(1)
    try:
        lecturer = re.search('(\w+\s+\w+)(?= \(course titular\))',body_text).group(1)
    except:
        lecturer = ""
    try:
        contact = re.search('(?<=contact hours )(.*?)(?= contact hours)',body_text).group(1)
    except:
        contact = ""
    
    
    #Course Content needs some special attention (Course material not always defined)
    try:
        content = re.search('(?<=Course Content )(.*?)(?= Course material)',body_text).group(1)
    except:
        content = re.search('(?<=Course Content )(.*?)(?= Additional info)',body_text).group(1)
    #remove unwanted chars
    content = content.replace('• ','').replace('*','')
    #remove rare occurence of <!-- --> comment in text
    try:
        html_comment = re.search('(<!--.*?-->)',content).group(1)
        if html_comment:
            content = content.replace(html_comment,'')
    except:
        pass
    
    ###
    
    #Lastly remap sem_value to 1, 2 or J     AND remap language to EN or NL
    semester = 'J' if 'and' in sem_value else ('1' if '1st' in sem_value else '2')
    language = 'EN' if 'English' in language else 'NL'
    
    #Save all course information in dict that can be pickled later
    courses_information[tag]['credits'] = credits
    courses_information[tag]['study_time'] = study
    courses_information[tag]['semester'] = semester
    courses_information[tag]['language'] = language
    courses_information[tag]['lecturer'] = lecturer
    courses_information[tag]['contact_hours'] = contact
    courses_information[tag]['content'] = content
    courses_information[tag]['study_time'] = str(float(courses_information[tag]['credits'])*30)
                                                 
    print('[%s] SEM %s - %s\n\t\t%s - %s - %s ECTS [%s,%s]\n%s\n\n%s\n\n' \
          %(tag, semester, courses_information[tag]['name'], \
            lecturer, language, credits, contact, study, courses_information[tag]['masters'], content))

[4019187DEW] SEM 1 - Introduction to Business
		Nikolay DENTCHEV - EN - 3 ECTS [18,90]
['VUBPho', 'VUBChem', 'VUBChem', 'VUBCivil', 'VUBElecMech', 'VUBElecMech', 'VUBElecMech', 'VUBElecMech']

The objective of this course is to provide an overview of the functional domains in business. This is achieved trough teaching classes and case development in groups. After cases are developed and discussed with other groups, students individually summarize the lessons learned with respect to the relation of various functional areas in business.The course covers the following topics: Introduction to business and economics Forms of business ownership Frameworks and tools for general management Introduction to human resource management, team building and communication Introduction to marketing Product design and development Operations management in manufacturing and service industries Introduction to financial accounting, financial analysis, and corporate finance (capital structure).


[4020324ENR]

[4012659ENR] SEM 2 - Sensors and Microsystem Electronics
		Maarten KUIJK - EN - 6 ECTS [36,160]
['VUBPho', 'VUBBioMed', 'VUBBioMed']

Chapter 1: Sensors en Actuators Type of Sensors en Actuators Calibration, Signal conditioning & Linearization Chapter 2: Elektronical Interfacing of Sensors en Actuators Transistor circuits Microcontroller concepts Interfacing using. PC-IO boards Driving array en reading out arrays Chapter 3: Elektronical transmission of data EMI, EMC, crosstalk in lumped en distributed systems Digital Transmission Analog Transmission Printed Circuit Board Design Chapter 4: Microsystems, real-world examples Systems on discrete components (example of an optical power monitoring system) Integrated chip systems (examples: ID-tag, PON receiver chip) Systems based on modules/components (examples ...) Systems having strong opto-electronic interaction (LCD-microdisplay, Photonic Mixer for 3D camera’s)


[4016818FNR] SEM 2 - Physics of Semiconductor Technologies and Devices
		Ro

[4012197FNR] SEM 2 - Photovoltaic Energy Conversion
		Decaan IR - EN - 4 ECTS [15,120]
['VUBPho']

This course in entirely taught at UGENT.Please visit the following link for more course info: http://studiegids.ugent.be/2014/EN/studiefiches/E900132.pdf.


[4016065FNR] SEM 2 - Quantum optics
		Decaan IR - EN - 4 ECTS [32,100]
['VUBPho']

Keywords: Quantum optics - Photonics - Coherence - Photon counting - Quantum interferences - Atom-light interaction - Single-photon sources - Two-photon sources Course structure: Part 1: Quantum statistical properties of light Chapter 1: Statistical properties of classical fields Chapter 2: Quantum theory of the electromagnetic field Chapter 3: Quantum states of light Chapter 4: Quantum theory of optical coherence Chapter 5: Quantum interferometry Part 2: Interaction of light with matter Chapter 6: Stucture of atoms and their interaction with light Chapter 7: Weak atom-light coupling Chapter 8: Strong atom-light coupling Part 3: Sources of quantum light

[4017020ENR] SEM 2 - Fault Detection and Isolation in Industrial Processes
		Michel KINNAERT - EN - 3 ECTS [18,80]
['VUBPho', 'VUBElec']

The course aims at mastering the basic tools for the design of model based systems for fault detection and isolation. The first part deals with the generation of fault indicators, also called residuals. The so-called parity space approach and the observer-based approach are successively developed in a deterministic framework and in a stochastic framework. The second part consists of the processing of the residuals to detect and isolate possible faults. To this end, statistical change detection/isolation algorithms are studied. Finally the use of parameter estimation methods is considered for fault detection.The valid fiche can be found at the following link: MATH-H503 Change the language to English in the dropdown menu on top of the page.


[4007427EER] SEM 1 - Measuring and Modelling of Nonlinear Systems
		Joannes SCHOUKENS - EN - 3 ECTS [18,90]
['V

[4019809ENR] SEM 1 - Advanced Measurement and Identification
		John LATAIRE - EN - 4 ECTS [24,110]
['VUBPho', 'VUBElec']

General goal of the course Engineers and scientists build models to understand, describe, predict and control the behaviour of the environment. In order to create these models it is necessary to combine the mathematical models with (noisy) measurements. In this advanced course, we will explore the consequences of having systems which do not satisfy the basic assumptions made when making frequency response function measurements. That is, we will learn how to deal with nonlinearities and time variations. Also, an introduction is given to the measurement of very high frequency signals and systems, and to the use of recursive identification techniques. In addition to a sound theoretical basis, we will also provide the students with hand on experiences in the labs. Short outline Measuring high frequency signals and systems Taking into account time variations and nonlinea

...

[4016363FNR] SEM 2 - Power System Engineering
		Claude MAUN - EN - 3 ECTS [24,85]
['VUBElecMech']

The valid fiche can be found at the following link : ELEC-H512. Change the language to English in the dropdown menu on top of the page.- Symmetrical components- Control of power systems- Stability of power systemsExercisesLabsAmount of hours for lectures : 1.5 ECTS (18h) ; exercises/labs/seminars : 1.5 ECTS (18h)


[4016362FNR] SEM 2 - Power System Protection
		Claude MAUN - EN - 3 ECTS [12,80]
['VUBElecMech']

The valid fiche can be found at the following link : ELEC-H510. Change the language to English in the dropdown menu on top of the page.Concepts of selective protection relaying for power systems. Protection of lines, cables, transformers, generators and motors. Wide area protection and system protection schemes.Course objectives : to give an overview of the present numerical relays capabilities for an efficient protection of electrical power apparatus.Teaching methods : Lectures 12

[4016327FNR] SEM 2 - Aircraft Conceptual Design
		Patrick HENDRICK - EN - 5 ECTS [24,130]
['VUBElecMech']

The valid fiche can be found at the following link : MECA-H508. Change the language to English in the dropdown menu on top of the page.This course is intended to focus on a first "system study" at the level of an aircraft for the students in aeronautical engineering. This course integrated a lot of matters that have been taught during the BA, the MA1 and the first part of the MA2 and these matters are now applied on a real case study integrating all of them.24 hours of theory are given with a description of the different steps into the design process of an aircraft. 36 hours are devoted to the aircraft design project of the student working in a design team of 3-4-5 students supervised by a leader (depending on the aircraft and the number of students).


[4016328FNR] SEM 2 - Avionics
		Marnix GOOSSENS - EN - 5 ECTS [30,125]
['VUBElecMech']

This course gives a descriptive introduct

[4016336FNR] SEM 2 - Computational Fluid Dynamics
		Gérard DEGREZ - EN - 3 ECTS [12,85]
['VUBElecMech']

-Navier-Stokes equations -the control volume method -basic schemes for convective terms -basic schemes for diffusive terms -Runge-Kutta solvers -multigrid During the exercises the use of a commercial CFD package, including meshgeneration and postprocessing, is explained (12 hours). The remaining 20 hours are used for a project, done independently by the student, in which a CFD package is used to solve some flow problems for which the results can be compared to reference results.The valid fiche for ULB can be found at the following link : MECA-H522. Change the language to English in the dropdown menu on top of the page.


[6009268FNR] SEM 1 - Introduction to nuclear physics and measurements
		Decaan IR - EN - 3 ECTS [25,85]
['VUBNucl']

Nuclear properties (nuclear radius; mass abundance of nuclides; nuclear binding energy; nuclear exited states) Elementary intro to nuclear models (dr

[6019825FNR] SEM 2 - Advanced courses of the nuclear fuel cycle
		Decaan IR - EN - 3 ECTS [24,80]
['VUBNucl']

Information available at the BNEN-website: http://bnen.sckcen.be/en/Programme sub 'Courses'


[6019826FNR] SEM J - Nuclear and radiological risk governance
		Decaan IR - EN - 3 ECTS [18,80]
['VUBNucl']

Information available at the BNEN-website: http://bnen.sckcen.be/en/Programme sub 'Courses'


[6009287FER] SEM J - Advanced course elective topic
		Peter BAETEN - EN - 3 ECTS [16,90]
['VUBNucl']

Information is available in the course sheet at the BNEN website: http://bnen.sckcen.be/en/Programme under the heading 'Courses'.


[4017610DNR] SEM 1 - Applied Statistics
		Olivier THAS - EN - 5 ECTS [15,135]
['VUBLand']

1. Descriptive statistics (means, medians, percentiles, …) 2. Some common distributions: normal, binomial, multinomial 3. Basics of statistical inference: confidence intervals and statistical hypothesis tests 4. Statistical tests for association in contingency tables

[4016684ENR] SEM J - Internship
		Decaan IR - EN - 5 ECTS [135,135]
['VUBLand']

under construction


[4004855FNR] SEM 1 - Groundwater modelling
		Marijke HUYSMANS - EN - 5 ECTS [12,125]
['VUBLand']

1) Introduction to groundwater modelling mathematics, basic parameters and variables, continuity equation, momentum equation, flow equations for different conditions, boundary conditions, approximate groundwater flow equations; 2) Finite difference techniques for groundwater modelling: steady and transient flow, numerical approximation of boundary conditions, matrix inversion techniques and iterative solvers, linear and non-linear problems, stability and convergence criteria; 3) Finite element techniques for groundwater modelling: basic principles and element types, triangular finite elements for 2D steady flow, numerical approximation of boundary conditions, matrix inversion techniques and iterative solvers, hexahedral finite element technique for 3D transient flow, iso-parametric element

### D) Pickle for later use

In [19]:
import os, sys, pickle

if not os.path.exists("courses"):
    os.mkdir("courses")

for tag in list(courses_information.keys()):
    with open('./courses/%s.pkl' % tag, 'wb') as f:
        pickle.dump(courses_information[tag], f, pickle.HIGHEST_PROTOCOL)
        
print('Pickled! :-)')

Pickled! :-)
